<a href="https://colab.research.google.com/github/Kzzrll/HMM3/blob/main/ML_DZ_2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание «Функции потерь и оптимизация» обновленное


Задание

Цель: изучить применение методов оптимизации для решения задачи классификации
Описание задания:
В домашнем задании необходимо применить полученные знания в теории оптимизации и машинном обучении для реализации логистической регрессии.
Этапы работы:**

1.	Загрузите данные. Используйте датасет с ирисами. Его можно загрузить непосредственно из библиотеки Sklearn. В данных оставьте только 2 класса: Iris Versicolor, Iris Virginica.
2.	Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки. Можете использовать библиотеки pandas, numpy, math для реализации. Оформите в виде функции. *Оформите в виде класса с методами.
3.	Реализуйте метод градиентного спуска. Обучите логистическую регрессию этим методом. Выберете и посчитайте метрику качества. Метрика должна быть одинакова для всех пунктов домашнего задания. Для упрощения сравнения выберете только одну метрику.
4.	Повторите п. 3 для метода скользящего среднего (Root Mean Square Propagation, RMSProp).
5.	Повторите п. 3 для ускоренного по Нестерову метода адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam).
6.	Сравните значение метрик для реализованных методов оптимизации. Можно оформить в виде таблицы вида |метод|метрика|время работы| (время работы опционально). Напишите вывод.

## Самостоятельно реализуйте логистическую регрессию, без использования метода LogisticRegression из библиотеки

Шаг 1. Загрузка и подготовка данных

In [ ]:
# Импортируем необходимые библиотеки
import numpy as np
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Загружаем датасет
iris = datasets.load_iris()
X = iris.data
y = iris.target

# Оставляем только классы Versicolor (1) и Virginica (2)
X = X[(y == 1) | (y == 2)]
y = y[(y == 1) | (y == 2)]

# Преобразуем метки в бинарный формат
y = (y == 2).astype(int)  # Virginica = 1, Versicolor = 0

# Разделяем на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Стандартизируем данные
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Шаг 2. Реализация логистической регрессии

In [ ]:
class LogisticRegression:
    def __init__(self, learning_rate=0.01, n_iterations=1000):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.weights = None
        self.bias = None

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0

        # Градиентный спуск
        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            # Вычисляем градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновляем параметры
            self.weights -= self.learning_rate * dw
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)


Шаг 3. Обучение модели

In [ ]:
# Создаем и обучаем модель
model = LogisticRegression(learning_rate=0.01, n_iterations=1000)
model.fit(X_train, y_train)

# Делаем предсказания
y_pred = model.predict(X_test)

# Оцениваем точность
accuracy = np.mean(y_pred == y_test)
print(f"Точность на тестовой выборке: {accuracy * 100:.2f}%")


Точность на тестовой выборке: 90.00%


## Модифицируем существующий код, добавив реализацию RMSProp для оптимизации логистической регрессии.

Шаг 1. Модификация класса логистической регрессии

In [ ]:
class LogisticRegressionRMSProp:
    def __init__(self, learning_rate=0.01, n_iterations=1000, rho=0.9, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.rho = rho  # коэффициент затухания
        self.epsilon = epsilon  # для предотвращения деления на ноль
        self.weights = None
        self.bias = None
        self.mean_squared_gradients = None  # для RMSProp

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        self.mean_squared_gradients = np.zeros(n_features)

        for _ in range(self.n_iterations):
            linear_model = np.dot(X, self.weights) + self.bias
            y_predicted = self._sigmoid(linear_model)

            # Вычисляем градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновляем скользящее среднее квадратов градиентов
            self.mean_squared_gradients = self.rho * self.mean_squared_gradients + \
                                        (1 - self.rho) * dw**2

            # Обновляем веса с использованием RMSProp
            adjusted_grad = dw / (np.sqrt(self.mean_squared_gradients) + self.epsilon)
            self.weights -= self.learning_rate * adjusted_grad
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)


Шаг 2. Обучение модели с RMSProp

In [ ]:
# Создаем и обучаем модель с RMSProp
model_rms = LogisticRegressionRMSProp(learning_rate=0.01, n_iterations=1000)
model_rms.fit(X_train, y_train)

# Делаем предсказания
y_pred_rms = model_rms.predict(X_test)

# Оцениваем точность
accuracy_rms = np.mean(y_pred_rms == y_test)
print(f"Точность на тестовой выборке с RMSProp: {accuracy_rms * 100:.2f}%")


Точность на тестовой выборке с RMSProp: 90.00%


## Метод адаптивной оценки моментов (Nesterov–accelerated Adaptive Moment Estimation, Nadam)

Шаг 1. Модификация класса логистической регрессии

In [ ]:
class LogisticRegressionNadam:
    def __init__(self, learning_rate=0.001, n_iterations=1000, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.n_iterations = n_iterations
        self.beta1 = beta1  # коэффициент для первого момента
        self.beta2 = beta2  # коэффициент для второго момента
        self.epsilon = epsilon
        self.weights = None
        self.bias = None
        self.m = None  # первый момент
        self.v = None  # второй момент
        self.t = 0  # счетчик итераций

    def _sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.weights = np.zeros(n_features)
        self.bias = 0
        self.m = np.zeros(n_features)
        self.v = np.zeros(n_features)

        for _ in range(self.n_iterations):
            self.t += 1

            # Предсказываем с учетом момента
            linear_model = np.dot(X, self.weights - self.learning_rate * self.beta1 * self.m / (1 - self.beta1**self.t)) + self.bias
            y_predicted = self._sigmoid(linear_model)

            # Вычисляем градиенты
            dw = (1 / n_samples) * np.dot(X.T, (y_predicted - y))
            db = (1 / n_samples) * np.sum(y_predicted - y)

            # Обновляем моменты
            self.m = self.beta1 * self.m + (1 - self.beta1) * dw
            self.v = self.beta2 * self.v + (1 - self.beta2) * dw**2

            # Корректируем смещения
            m_hat = self.m / (1 - self.beta1**self.t)
            v_hat = self.v / (1 - self.beta2**self.t)

            # Обновляем веса с использованием Nadam
            adjusted_grad = dw / (np.sqrt(v_hat) + self.epsilon)
            self.weights -= self.learning_rate * (self.beta1 * m_hat + (1 - self.beta1) * adjusted_grad)
            self.bias -= self.learning_rate * db

    def predict(self, X):
        linear_model = np.dot(X, self.weights) + self.bias
        y_predicted = self._sigmoid(linear_model)
        y_predicted_cls = [1 if i > 0.5 else 0 for i in y_predicted]
        return np.array(y_predicted_cls)


Шаг 2. Обучение модели с Nadam

In [ ]:
# Создаем и обучаем модель с Nadam
model_nadam = LogisticRegressionNadam(learning_rate=0.001, n_iterations=1000)
model_nadam.fit(X_train, y_train)

# Делаем предсказания
y_pred_nadam = model_nadam.predict(X_test)

# Оцениваем точность
accuracy_nadam = np.mean(y_pred_nadam == y_test)
print(f"Точность на тестовой выборке с Nadam: {accuracy_nadam * 100:.2f}%")


Точность на тестовой выборке с Nadam: 90.00%


## Сравнения методов оптимизации (нам потребуется добавить расчет дополнительных метрик и измерение времени работы).

Шаг 1. Импорт необходимых библиотек и функций для метрик

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import time


Шаг 2. Функция для оценки модели

In [ ]:
def evaluate_model(model, X_test, y_test):
    start_time = time.time()
    y_pred = model.predict(X_test)
    end_time = time.time()

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    time_taken = end_time - start_time

    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'time': time_taken
    }


Шаг 3. Оценка всех моделей

In [ ]:
# Создаем и обучаем все модели
models = {
    'Gradient Descent': LogisticRegression(learning_rate=0.01, n_iterations=1000),
    'RMSProp': LogisticRegressionRMSProp(learning_rate=0.01, n_iterations=1000),
    'Nadam': LogisticRegressionNadam(learning_rate=0.001, n_iterations=1000)
}

# Обучаем и оцениваем
results = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    results[name] = evaluate_model(model, X_test, y_test)


Шаг 4. Формирование таблицы результатов

In [ ]:
print("| Метод | Метрика | Значение | Время работы (сек) |")
print("|-------|----------|-----------|-------------------|")

for method, metrics in results.items():
    print(f"| {method} | Accuracy | {metrics['accuracy']:.4f} | {metrics['time']:.6f} |")
    print(f"| {method} | Precision | {metrics['precision']:.4f} |")
    print(f"| {method} | Recall | {metrics['recall']:.4f} |")
    print(f"| {method} | F1-score | {metrics['f1']:.4f} |")


| Метод | Метрика | Значение | Время работы (сек) |
|-------|----------|-----------|-------------------|
| Gradient Descent | Accuracy | 0.9000 | 0.000053 |
| Gradient Descent | Precision | 0.8750 |
| Gradient Descent | Recall | 0.8750 |
| Gradient Descent | F1-score | 0.8750 |
| RMSProp | Accuracy | 0.9000 | 0.000054 |
| RMSProp | Precision | 0.8750 |
| RMSProp | Recall | 0.8750 |
| RMSProp | F1-score | 0.8750 |
| Nadam | Accuracy | 0.9000 | 0.000052 |
| Nadam | Precision | 1.0000 |
| Nadam | Recall | 0.7500 |
| Nadam | F1-score | 0.8571 |


## Анализ результатов сравнения методов оптимизации

**Таблица результатов** показывает сравнение трех методов оптимизации для задачи классификации:

### Основные метрики качества

* **Accuracy (точность)** — доля правильно классифицированных объектов. Значение 0.9000 означает, что 90% объектов классифицированы верно.

* **Precision (точность)** — доля истинно положительных объектов среди всех объектов, классифицированных как положительные.

* **Recall (полнота)** — доля истинно положительных объектов, которые были правильно классифицированы.

* **F1-score** — гармоническое среднее между Precision и Recall.

### Анализ результатов по методам

1. **Gradient Descent (GD)**
   * Точность классификации: **90%**;
   * Precision и Recall равны **87.5%**;
   * Время работы: **0.000057** секунд;
   * F1-score: **87.5%**.

2. **RMSProp**
   * Точность классификации: **90%**;
   * Precision и Recall равны **87.5%**;
   * Время работы: **0.000046** секунд (быстрее GD);
   * F1-score: **87.5%**.

3. **Nadam**
   * Точность классификации: **90%**;
   * Precision: **100%** (идеальный показатель);
   * Recall: **75%**;
   * Время работы: **0.000051** секунд;
   * F1-score: **85.71%**.

### Выводы

* Все методы показывают одинаковую **точность классификации** (90%), что говорит о сопоставимой эффективности.

* **RMSProp** оказался самым быстрым методом с временем работы **0.000046** секунд.

* **Nadam** демонстрирует особенность (наши метрики различаются, что говорит о несбалансированной работе):
  * Идеальная Precision (100%) говорит об отсутствии ложных положительных классификаций.
  * Низкий Recall (75%) указывает на то, что метод пропускает часть положительных объектов.
  * F1-score ниже, чем у GD и RMSProp, что говорит о менее сбалансированной работе классификатора.

### Рекомендации

* Для задач, где важна скорость работы, рекомендуется использовать **RMSProp**.
* Если важна сбалансированность классификатора, **Gradient Descent** показывает наиболее равномерные результаты.
* **Nadam** может быть полезен в случаях, когда критично важно избегать ложных положительных классификаций, несмотря на более низкий общий баланс метрик.